# RandAugment on Wide ResNet 28 10

# 1. Environment Setup
## 1.0. Check GPU

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


## 1.1. Import

In [2]:
import augmentation, RandAugment, train
import torch, torchvision
import torchvision.transforms as transforms

# Ignore Warning
import warnings
warnings.filterwarnings(action='ignore')

# 2. Data Preprocessing

In [3]:
N, M, p0, p1 = 3, 5, 1, 1

batch_size = 128
cifar_dir = '../../../../data/OCR/Jason/cifar10/'

transform_functions = ['Identity',
                       'AutoContrast', 'Brightness', 'Color', 'Contrast', 'Equalize', 'Posterize', 'Sharpness', 'Solarize', 
                       'Rotate', 'ShearX', 'ShearY', 'TranslateX', 'TranslateY']

In [4]:
pre_train_transform = [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()]
post_train_transform = [augmentation.Cutout(1, 8), transforms.ToTensor(), 
                        transforms.Normalize(mean=[0.4921, 0.4828, 0.4474], std=[0.1950, 0.1922, 0.1940])]

train_transform = RandAugment.RandAug(N, M, p0, p1, pre_train_transform, post_train_transform, transform_functions)

test_transform = transforms.Compose([transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.4921, 0.4828, 0.4474], std=[0.1950, 0.1922, 0.1940])])

train_dataset = torchvision.datasets.CIFAR10(cifar_dir, train=True, download=False, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=False)

test_dataset = torchvision.datasets.CIFAR10(cifar_dir, train=False, download=False, transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=False)

# 3. Train Model

In [5]:
depth = 28
widen_factor = 10
dropout_rate = 0.3
num_classes = 10

gpu_id = 7
count_freq = 60

epochs = 201

lr = 0.1
momentum = 0.9
weight_decay = 0.0005
nesterov = True
milestones = [60, 120, 160]

In [6]:
print('*' * 60)
print('   Augmentation Number : {}, Magnitude : {}, P0 : {}, P1 : {}'.format(N, M, p0, p1))
print('*' * 60)
print()

best_accs = []
for i in range(5):
    WRN_28_10 = train.Wide_ResNet_NoPrint(depth, widen_factor, dropout_rate, num_classes, gpu_id, count_freq)
    WRN_28_10.train(train_loader, test_loader, epochs, lr, momentum, weight_decay, nesterov, milestones)
    
    best_accs.append(WRN_28_10.best_acc)
    print('Trial : {} - Best Acc : {}'.format(i+1, best_accs[-1]))
    
print()
print(best_accs)
print(sum(best_accs)/len(best_accs))

************************************************************
   Augmentation Number : 3, Magnitude : 5, P0 : 1, P1 : 1
************************************************************

Trial : 1 - Best Acc : 97.08
Trial : 2 - Best Acc : 97.21
Trial : 3 - Best Acc : 97.26
Trial : 4 - Best Acc : 97.25
Trial : 5 - Best Acc : 97.16

[97.08, 97.21, 97.26, 97.25, 97.16]
97.19200000000001
